In [53]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

In [2]:
obwody = pd.read_csv('./obwody_glosowania_utf8.csv', sep=';')

In [3]:
obwody = obwody[['Gmina','TERYT gminy','Numer', 'Kod pocztowy']]

In [4]:
obwody = obwody.dropna()

In [5]:
obwody.head(2)

,Gmina,TERYT gminy,Numer,Kod pocztowy
0,m. Bolesławiec,20101.0,1,59-700
1,m. Bolesławiec,20101.0,2,59-700


# Klastrowanie

Z artykułu:
1. Początkowe grupowanie oparto na pierwszych dwóch cyfrach kodu pocztowego (np. „30” dla obszaru Krakowa).
2. Jeżeli powstała grupa zawierała od 10 do 16 komisji, została zaakceptowana bez zmian.
3. Grupy liczące mniej niż 10 komisji odłożono do późniejszego łączenia.
4. Grupy przekraczające 16 komisji dzielono rekurencyjnie, dodając kolejne cyfry kodu
    pocztowego (np. z „30” → „301” → „3011” i dalej, aż do pełnych pięciu cyfr).
5. Pozostałe małe grupy łączono z najbliższymi sąsiadami mającymi ten sam prefiks, przy czym
    priorytetem była ciągłość przestrzenna i zrównoważona liczebność grup.
    W odróżnieniu od wcześniejszego podejścia, które dopuszczało grupy o wielkości 10–25 komisji,
    niniejsze badanie przyjęło węższy zakres docelowy: od 10 do 16 komisji na grupę
   
   
Komentarz: Algorytm z artykułu nie jest deterministyczny (niejednoznacznie wskazuje co się dzieje z grupami liczącymi mniej niż 10 komisji do ponownego połączenia) nie da się go więc odtworzyć na podstawie jego opisu. Dlatego właśnie rzetelne artykułu zawierają kod źródłowy umożliwiający reprodukcję.

In [6]:
import pandas as pd
from collections import defaultdict

def assign_groups(df: pd.DataFrame,
                  code_col: str = "kod",
                  min_size: int = 10,
                  max_size: int = 16,
) -> pd.DataFrame:
    """
    Zwraca kopię df z nową kolumną `group_id` zawierającą numer grupy
    spełniającej warunki 10 ≤ n ≤ 16.
    """

    # 0. Normalizacja kodu do stałej długości 5 znaków (str)
    work = df.copy()
    work["_code_str"] = work[code_col].astype(str).str.zfill(5)

    accepted = {}                # {prefix: list[index]}
    small     = {}               # prefixy < min_size (do późniejszego łączenia)

    # 1. Rekurencyjne dzielenie dużych grup
    def split(prefix: str, idxs: list[int], depth: int):
        size = len(idxs)

        # a) gotowa grupa
        if min_size <= size <= max_size or depth == 5:
            accepted[prefix] = idxs
            return
        # b) za mała – zapisz do późniejszego łączenia
        if size < min_size:
            small[prefix] = idxs
            return
        # c) za duża – dziel głębiej
        next_depth = depth + 1
        sub_prefixes = work.loc[idxs, "_code_str"].str[:next_depth]
        for sub_pref, sub_idxs in work.loc[idxs].groupby(sub_prefixes).groups.items():
            split(sub_pref, list(sub_idxs), next_depth)

    # start od 2-cyfrowych
    for pref2, grp in work.groupby(work["_code_str"].str[:2]).groups.items():
        split(pref2, list(grp), depth=2)

    # 2. Łączenie małych grup w obrębie tych samych 2-cyfrowych prefiksów
    buckets = defaultdict(list)          # {pref2: [(pref, idxs), ...]}
    for p, idxs in small.items():
        buckets[p[:2]].append((p, idxs))

    for pref2, lst in buckets.items():
        # uporządkuj rosnąco wg wartości prefiksu → przybliżenie "sąsiedztwa"
        lst.sort(key=lambda x: int(x[0]))
        buf_idx, buf_pref = [], []

        for p, idxs in lst:
            buf_idx.extend(idxs)
            buf_pref.append(p)

            if len(buf_idx) >= min_size:
                # jeśli przypadkiem > max_size, tnij w kawałki
                while len(buf_idx) > max_size:
                    accepted[f"{pref2}_{len(accepted)}"] = buf_idx[:max_size]
                    buf_idx = buf_idx[max_size:]
                accepted["+".join(buf_pref)] = buf_idx
                buf_idx, buf_pref = [], []

        # resztka < 10 – dociągamy do ostatniej grupy z tego prefiksu
        if buf_idx:
            *_, last_key = [k for k in accepted.keys() if k.startswith(pref2)]
            accepted[last_key].extend(buf_idx)

    # 3. Ostateczne przypisanie numerów
    idx2gid = {}
    for gid, (_, lst) in enumerate(accepted.items()):
        for i in lst:
            idx2gid[i] = gid

    work["group_id"] = work.index.map(idx2gid)

    return work.drop(columns="_code_str")

In [7]:
obwody['kod'] = obwody['Kod pocztowy'].str.replace('-','').astype(int) 

obwody = assign_groups(obwody, code_col='kod')

Z artykułu:

"Większość grup spełniła założony docelowy rozmiar: 1 386 grup (62,8%) zawierało od 10 do 16
komisji, a 2 017 grup (91,3%) zawierało od 6 do 30 komisji. Większe grupy zazwyczaj odpowiadały 5
obszarom miejskim — na przykład takim jak Toruń czy Włocławek, gdzie pojedynczy kod pocztowy
obejmował całe miasto"

In [8]:
# między 10 a 16
sum((obwody['group_id'].value_counts()>=10) & (obwody['group_id'].value_counts()<=16))

1223

In [9]:
# między 6 a 30
sum((obwody['group_id'].value_counts()>=6) & (obwody['group_id'].value_counts()<=30))

1925

In [10]:
# duże grupy
obwody[obwody['group_id']==2150]

,Gmina,TERYT gminy,Numer,Kod pocztowy,kod,group_id
4100,m. Toruń,46301.0,1,87-100,87100,2150
4101,m. Toruń,46301.0,2,87-100,87100,2150
4102,m. Toruń,46301.0,3,87-100,87100,2150
4103,m. Toruń,46301.0,4,87-100,87100,2150
4104,m. Toruń,46301.0,5,87-100,87100,2150
...,...,...,...,...,...,...
4221,m. Toruń,46301.0,122,87-100,87100,2150
4222,m. Toruń,46301.0,123,87-100,87100,2150
4223,m. Toruń,46301.0,124,87-100,87100,2150
4224,m. Toruń,46301.0,125,87-100,87100,2150


In [11]:
# usun grupy mniej niz 3-komisjowe
obwody['liczba komisji w grupie']  = obwody.groupby('group_id').transform('count')['kod']
obwody = obwody[obwody['liczba komisji w grupie']>3]

In [12]:
len(obwody)

30553

# Wyniki głosowania

In [13]:
p = pd.read_csv('./protokoly_po_obwodach_utf8.csv', sep=';')
d = pd.read_csv('./protokoly_po_obwodach_w_drugiej_turze_utf8.csv', sep=';')


In [14]:
keep = ['Teryt Gminy', 'Nr komisji', 'Gmina', 'TRZASKOWSKI Rafał Kazimierz', 'NAWROCKI Karol Tadeusz']

In [15]:
p = p[keep].dropna()
d = d[keep].dropna()

In [16]:
p.head(2)

,Teryt Gminy,Nr komisji,Gmina,TRZASKOWSKI Rafał Kazimierz,NAWROCKI Karol Tadeusz
0,20101.0,1,m. Bolesławiec,361.0,287.0
1,20101.0,2,m. Bolesławiec,381.0,228.0


In [17]:
# polacz 1 i 2 tura

In [18]:
df = pd.merge(
    left=p, 
    right=d,
    how='left',
    left_on=['Teryt Gminy', 'Nr komisji'],
    right_on=['Teryt Gminy', 'Nr komisji'],
    suffixes = ('_1', '_2')
)

In [19]:
# polacz z obwodami

In [20]:
df = pd.merge(
    left=df, 
    right=obwody,
    how='left',
    left_on=['Teryt Gminy', 'Nr komisji'],
    right_on=['TERYT gminy', 'Numer'],
)


In [21]:
from copy import copy
df = copy(df.dropna())

# Reprodukcja Analizy z artykułu dla obu kandydatów

## 1. Nadmierne poparcie dla Karola Nawrockiego (względem mediany w ramach lokalnej grupy)

In [22]:
kandydat_A = "TRZASKOWSKI Rafał Kazimierz"
kandydat_B = "NAWROCKI Karol Tadeusz"

In [23]:
# mediana w grupie
df[kandydat_A + '_mediana_2'] = df.groupby('group_id')[kandydat_A+'_2'].transform('median')
df[kandydat_B + '_mediana_2'] = df.groupby('group_id')[kandydat_B+'_2'].transform('median')

In [24]:
# MAD w grupie
from scipy.stats import median_abs_deviation
def mad(series):
    """Median Absolute Deviation"""
    return median_abs_deviation(series)

df[kandydat_A + "_MAD_2"] = df.groupby("group_id")[kandydat_A+'_2'].transform(mad)
df[kandydat_B + "_MAD_2"] = df.groupby("group_id")[kandydat_B+'_2'].transform(mad)

In [25]:
# wspolczynnik k

In [26]:
df[kandydat_A +'_k_analiza_1'] = (df[kandydat_A +'_2'] - df[kandydat_A + '_mediana_2'])/df[kandydat_A + '_MAD_2']

In [27]:
df[kandydat_B +'_k_analiza_1'] = (df[kandydat_B +'_2'] - df[kandydat_B + '_mediana_2'])/df[kandydat_B + '_MAD_2']

In [28]:
k = 3

In [29]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_1']>k)

('TRZASKOWSKI Rafał Kazimierz', 2794)

In [30]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 2015)

In [31]:
k = 2

In [32]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_1']>k)

('TRZASKOWSKI Rafał Kazimierz', 4551)

In [33]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 3750)

### Wynik:
Dla k=2, takich grup, w których "za duże" poparcie ma Nawrocki jest 3750, a Trzaskowski 4551.

## 2. Nadmierny względny wzrost poparcia dla Karola Nawrockiego między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Rafała Trzaskowskiego w tej samej grupie lokalnej;


In [34]:
# wzgledny wzrost między pierwszą a drugą turą
df[kandydat_A+ '_wzrost'] = df[kandydat_A+'_2']/df[kandydat_A+'_1']
df[kandydat_B+ '_wzrost'] = df[kandydat_B+'_2']/df[kandydat_B+'_1']

In [35]:
# roznica wzglednego roz_wzru miedzy kandydatami
df['roz_wzr_' + kandydat_A] =  df[kandydat_A + '_wzrost']  - df[kandydat_B + '_wzrost'] 
df['roz_wzr_' + kandydat_B] =  df[kandydat_B + '_wzrost']  - df[kandydat_A + '_wzrost'] 

In [36]:
# mediana i mad różnicy względnego wzrostu poparcia

df['roz_wzr_' + kandydat_A +'_mediana'] = df.groupby('group_id')['roz_wzr_' + kandydat_A].transform('median')
df['roz_wzr_' + kandydat_A + '_MAD'] =    df.groupby('group_id')['roz_wzr_' + kandydat_A].transform(mad)

In [37]:
df['roz_wzr_' + kandydat_B +'_mediana'] = df.groupby('group_id')['roz_wzr_' + kandydat_B].transform('median')
df['roz_wzr_' + kandydat_B + '_MAD'] =    df.groupby('group_id')['roz_wzr_' + kandydat_B].transform(mad)

In [38]:
df[kandydat_A +'_k_analiza_2'] = (df['roz_wzr_' + kandydat_A] - df['roz_wzr_' + kandydat_A +'_mediana'])/df['roz_wzr_' + kandydat_A + '_MAD']

In [39]:
df[kandydat_B +'_k_analiza_2'] = (df['roz_wzr_' + kandydat_B] - df['roz_wzr_' + kandydat_B +'_mediana'])/df['roz_wzr_' + kandydat_B + '_MAD']

In [40]:
k = 3

In [41]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_2']>k)

('TRZASKOWSKI Rafał Kazimierz', 2106)

In [42]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 2015)

In [43]:
k = 2

In [44]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_2']>k)

('TRZASKOWSKI Rafał Kazimierz', 3552)

In [45]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 3750)

"Nadmierny" względny wzrost poparcia dla Trzaskowskiego odnotowano w 3552 komisjach
"Nadmierny" względny wzrost poparcia dla Nawrockiego odnotowano w 3750 komisjach

### 3 Komisje, w których Nawrocki uzyskał więcej głosów niż Trzaskowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Trzaskowskiego;

In [46]:
# wieksza mediana w grupie
df['wieksza_mediana_' + kandydat_A] = (df[kandydat_A + '_mediana_2'] >  df[kandydat_B + '_mediana_2']).astype(bool)
df['wieksza_mediana_' + kandydat_B] = (df[kandydat_B + '_mediana_2'] >  df[kandydat_A + '_mediana_2']).astype(bool)

In [47]:
df_wieksza_mediana_kandydat_A = df[df['wieksza_mediana_' + kandydat_A]]
df_wieksza_mediana_kandydat_B = df[df['wieksza_mediana_' + kandydat_B]]

In [48]:
kandydat_A, sum(df_wieksza_mediana_kandydat_A[kandydat_B+'_2']>df_wieksza_mediana_kandydat_A[kandydat_A+'_2'])

('TRZASKOWSKI Rafał Kazimierz', 1843)

In [49]:
kandydat_B, sum(df_wieksza_mediana_kandydat_B[kandydat_A+'_2']>df_wieksza_mediana_kandydat_B[kandydat_B+'_2'])

('NAWROCKI Karol Tadeusz', 2608)

W grupach, w których większą medianę miał Trzaskowski, 1843 razy wyższy wyniki uzyskał Nawrocki.

W grupach, w których większą medianę miał Nawrocki, 2608 razy wyższy wynik uzyskał Trzaskowski.

## 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [50]:
kandydat_A, sum(df[kandydat_A + '_2']<df[kandydat_A + '_1'])

('TRZASKOWSKI Rafał Kazimierz', 128)

In [51]:
kandydat_B, sum(df[kandydat_B + '_2']<df[kandydat_B + '_1'])

('NAWROCKI Karol Tadeusz', 112)

W 128 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

W 112 komisjach Nawrocki uzyskał mniej głosó w drugiej turze niż w pierwszej.

In [52]:
df[df['group_id']==1541]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz
1871,26101.0,12,m. Jelenia Góra,226.0,102.0,m. Jelenia Góra,368.0,194.0,m. Jelenia Góra,26101.0,12.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,-1.548023,-3.169492,1.628319,1.901961,-0.273642,0.273642,-0.182338,0.101954,0.182338,0.101954,-0.895545,0.895545,True,False
1872,26101.0,13,m. Jelenia Góra,321.0,133.0,m. Jelenia Góra,496.0,228.0,m. Jelenia Góra,26101.0,13.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,-0.824859,-2.593220,1.545171,1.714286,-0.169114,0.169114,-0.182338,0.101954,0.182338,0.101954,0.129703,-0.129703,True,False
1873,26101.0,14,m. Jelenia Góra,384.0,171.0,m. Jelenia Góra,574.0,283.0,m. Jelenia Góra,26101.0,14.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,-0.384181,-1.661017,1.494792,1.654971,-0.160179,0.160179,-0.182338,0.101954,0.182338,0.101954,0.217343,-0.217343,True,False
1874,26101.0,15,m. Jelenia Góra,304.0,195.0,m. Jelenia Góra,445.0,321.0,m. Jelenia Góra,26101.0,15.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,-1.112994,-1.016949,1.463816,1.646154,-0.182338,0.182338,-0.182338,0.101954,0.182338,0.101954,0.000000,0.000000,True,False
1875,26101.0,16,m. Jelenia Góra,447.0,242.0,m. Jelenia Góra,642.0,389.0,m. Jelenia Góra,26101.0,16.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,0.000000,0.135593,1.436242,1.607438,-0.171196,0.171196,-0.182338,0.101954,0.182338,0.101954,0.109282,-0.109282,True,False
1876,26101.0,17,m. Jelenia Góra,523.0,239.0,m. Jelenia Góra,738.0,433.0,m. Jelenia Góra,26101.0,17.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,0.542373,0.881356,1.411090,1.811715,-0.400626,0.400626,-0.182338,0.101954,0.182338,0.101954,-2.141046,2.141046,True,False
1877,26101.0,18,m. Jelenia Góra,713.0,253.0,m. Jelenia Góra,984.0,440.0,m. Jelenia Góra,26101.0,18.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,1.932203,1.000000,1.380084,1.739130,-0.359046,0.359046,-0.182338,0.101954,0.182338,0.101954,-1.733221,1.733221,True,False
1878,26101.0,19,m. Jelenia Góra,594.0,220.0,m. Jelenia Góra,829.0,386.0,m. Jelenia Góra,26101.0,19.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,1.056497,0.084746,1.395623,1.754545,-0.358923,0.358923,-0.182338,0.101954,0.182338,0.101954,-1.732007,1.732007,True,False
1879,26101.0,20,m. Jelenia Góra,734.0,262.0,m. Jelenia Góra,1044.0,483.0,m. Jelenia Góra,26101.0,20.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,2.271186,1.728814,1.422343,1.843511,-0.421168,0.421168,-0.182338,0.101954,0.182338,0.101954,-2.342535,2.342535,True,False
1880,26101.0,21,m. Jelenia Góra,596.0,286.0,m. Jelenia Góra,819.0,416.0,m. Jelenia Góra,26101.0,21.0,58-506,58506.0,1541.0,13.0,642.0,381.0,177.0,59.0,1.000000,0.593220,1.374161,1.454545,-0.080384,0.080384,-0.182338,0.101954,0.182338,0.101954,1.000000,-1.000000,True,False
